In [ ]:
# If using colab 
# !pip install "pyGTFSHandler[osm,plot] @ git+https://github.com/CityScope/pyGTFSHandler.git"
# !pip install matplotlib mapclassify folium

In [ ]:
import os

from pyGTFSHandler.feed import Feed
from pyGTFSHandler.downloaders.mobility_database import MobilityDatabaseClient
from pyGTFSHandler.utils import get_geographic_suggestions_from_string
from pyGTFSHandler.utils import get_city_geometry
import pyGTFSHandler.plot_helper as plot_helper
import pyGTFSHandler.gtfs_checker as gtfs_checker
import pyGTFSHandler.processing_helper as processing_helper
from datetime import datetime, date, timedelta, time
import pandas as pd
import polars as pl 
import geopandas as gpd
import numpy as np
from shapely import wkt

import matplotlib.pyplot as plt

In [ ]:
def df_to_stop_gdf(df):
    if isinstance(df,pl.LazyFrame):
        df = df.collect()
    if isinstance(df, pl.DataFrame):
        df = df.to_pandas()

    df = gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df['stop_lon'],df['stop_lat']),crs=4326)
    return df 

In [ ]:
city_name = "Cambridge, Massachusetts, USA"
download_buffer = 2000 # In meters

In [ ]:
# Name of the folder with results
city_filename = gtfs_checker.normalize_string(city_name)

# Area of interest as a Polygon
aoi = get_city_geometry(city_name)

# Area used for downloading data
aoi_download = aoi.to_crs(aoi.estimate_utm_crs()).buffer(download_buffer)

# Country, State and Municipality of your AOI
geo_suggestions = get_geographic_suggestions_from_string(city_name)

geo_suggestions

In [ ]:
# Save all files in a dir with the name of the city
os.makedirs(city_filename,exist_ok=True)

In [ ]:
# Request your refresh token here: https://mobilitydatabase.org/ 
refresh_token = ''
api = MobilityDatabaseClient(refresh_token)

#### Search feeds

In [ ]:
feeds = api.search_gtfs_feeds(
    country_code=geo_suggestions['country_codes'],
    # This info is not always in the feeds metadata. Comment this if you did not find all feeds.
    subdivision_name=geo_suggestions['subdivision_names'],
    # This info is not always in the feeds metadata. Comment this if you did not find all feeds.
    municipality=geo_suggestions['municipalities'],
    # Set to True if you only want official feeds otherwise set to None (False means only unofficial)
    # is_official=True, 
    # You could comment the rest of search args and use only aoi
    # Sometimes the API seems to not do this very well as the metadata is often wrong.
    # aoi=aoi_download, 
)

for f in feeds:
    print(f['provider'])

#### Download feeds

In [ ]:
orig_file_paths = api.download_feeds(
    feeds=feeds,
    download_folder=city_filename+"/orig_gtfs_files",
    overwrite=False
)

In [ ]:
# A) Do not fix the gtfs. Feed has a fast basic gtfs fixer but does not log all the errors.

file_paths = orig_file_paths

# B) Check and fix the gtfs files (This takes a few minutes). Set check_files = False in Feed to load faster

# file_paths = []
# for f in orig_file_paths:
#     filename = os.path.splitext(os.path.basename(f))[0]
#     if os.path.isdir(os.path.join(city_filename,"gtfs_files",filename)):
#         file_paths.append(os.path.join(city_filename,"gtfs_files",filename))
#     else:
#         file_paths.append(gtfs_checker.preprocess_gtfs(f,city_filename+"/gtfs_files"))

### Feed object

This object contains all methods you can do with the GTFS and a dataframe with all info

Any method of the Feed object return a polars dataframe. If you want the dataframe as pandas to .to_pandas()

In [ ]:
gtfs = Feed(
    file_paths,
    aoi=aoi,
    # Group stops into same that are less than x meters apart. This updates the parent_station column
    stop_group_distance=100, 
    start_date=date.today(), # If None take the feed min date
    end_date=date.today() + timedelta(days=30), # If None take the feed max date
)

### Service Intensity

This is the number of vehicles that arrive at each stop every day multiplied by the number of stops:

$\text{Service Intensity} = (\text{Number of vehicles per stop}) \times (\text{Number of stops})$

In [ ]:
service_intensity = gtfs.get_service_intensity_in_date_range(
    start_date=None, # If None take the feed min date
    end_date=None, # If None take the feed max date
    date_type=None, # Could be something like 'holiday', 'weekday', or 'monday' to only consider some dates from the range.
    by_feed=True
)
service_intensity = service_intensity.to_pandas()
plot_helper.service_intensity(service_intensity)

Select the most representative business day in a date range

In [ ]:
idx = processing_helper.most_frequent_row_index(service_intensity)
selected_day = service_intensity.iloc[idx]['date'].to_pydatetime()
selected_day

## Basic parameters of your analysis

In [ ]:
# date and time
date = selected_day # date in this format date(year=2025,month=4,day=1)
start_time = time(hour=8)
end_time = time(hour=20)

route_types = 'all' # Valid values are 'tram' 'subway' 'rail' 'bus' 'ferry' 'cable_car' 'gondola' 'funicular' 
# or any list combining those like ['rail', 'subway']

stop_id = "parent_station" # Use the stop groups created with arg stop_group_distance in Feed to group neraby stops into one
# You could choose 'stop_id' too

## Point data

### Speed

At every stop and by every route 

Speeds are in *km/h*

In [ ]:
stop_speed_df = gtfs.get_mean_speed_at_stops(
    date=selected_day,
    start_time=start_time,
    end_time=end_time,
    route_types = route_types,
    by = "route_id", # Speed is computed for every 'trip_id' and grouped by this column with the how method
    at = stop_id, # Compute speed for every 'parent_station' 'stop_id' or 'route_id'
    how="mean", # How to group individual trip speeds 'mean' 'max' or 'min'
    direction="both", # Compute speed in 'forward' 'backward' or 'both' directions (walking n_stops in direction)
    time_step=15, # Number of minutes to move to compute speed
).to_pandas()
stop_speed_df = gtfs.add_stop_coords(stop_speed_df)
stop_speed_df = gtfs.add_route_names(stop_speed_df)
stop_speed_df = df_to_stop_gdf(stop_speed_df)
stop_speed_df = stop_speed_df.sort_values(stop_id).reset_index(drop=True)
stop_speed_df = stop_speed_df[stop_speed_df.geometry.is_valid]
stop_speed_df 

In [ ]:
# Save the speed_df GeoDataFrame to open in QGis 
stop_speed_df.to_file(city_filename+"/stop_speed.gpkg")

Only best route at every stop

In [ ]:
idx = stop_speed_df.groupby(stop_id)["speed"].idxmax()
idx = idx.dropna()

best_stop_speed_df = stop_speed_df.loc[idx]
best_stop_speed_df

In [ ]:
# Save the speed_df GeoDataFrame to open in QGis 
best_stop_speed_df.to_file(city_filename+"/stop_speed_best.gpkg")

In [ ]:
m = best_stop_speed_df[
    [
        stop_id,
        "speed",
        "stop_name",
        "route_name",
        "route_type_text",
        "geometry",
    ]
].explore(
    column=f"speed",
    cmap="RdYlGn",
    vmin=10,
    vmax=30,
    style_kwds={
        "color": "black",  # Border color
        "weight": 1,  # Border thickness
        "opacity": 1.0,  # Border opacity
        "fillOpacity": 1,
        "radius": 6,
    },
)
m

In [ ]:
# Mean speed by route 
route_speed = (
    stop_speed_df
    .groupby('route_id')
    .apply(lambda g: (g['speed'] * g['distance_weight']).sum() / g['distance_weight'].sum())
    .reset_index(name='speed')
)
route_speed = gtfs.add_route_names(route_speed)
route_speed

In [ ]:
route_speed.to_csv(city_filename+"/route_speeds.csv")

In [ ]:
# Mean speed in whole system 
system_speed = float(
    np.nansum(stop_speed_df['distance_weight'] * stop_speed_df['speed']) /
    np.nansum(stop_speed_df['distance_weight'])
)

print(f"The average speed in the system is {round(system_speed,ndigits=2)} km/h")

### Average waiting time (interval) at stops

Interval is in *minutes*

In this example we use the *shape_direction* mode 

- Uses direction (shape_direction) in degrees from north that every 'trip_id' is pointing to computed at every stop and every trip
- Creates 'n_divisions' * 2 groups (*2 to get outbound and inbound directions independently) by clustering the trip shape directions 
- If how = 'best' means the interval is computed only for the best of all divisions at every stop 

In [ ]:
stop_interval_df = gtfs.get_mean_interval_at_stops(
    date=selected_day,
    start_time=start_time,
    end_time=end_time,
    route_types=route_types, 
    by = "shape_direction", # Interval is computed for all 'trip_id' grouped by this column and sorted by 'departure_time'
    at = stop_id, # Where to compute the interval 'stop_id' 'parent_station'
    how = "best", 
    # 'best' pick the route with best interval, 
    # 'mean' Combine all intervals of all routes, 
    # 'all' return results per stop and route
    n_divisions=1, # Number of divisions for by = 'shape_direction'
    # mix_directions=False # For how = 'mean' do you want to consider both directions as different routes?
).to_pandas()
stop_interval_df = gtfs.add_stop_coords(stop_interval_df)
stop_interval_df = gtfs.add_route_names(stop_interval_df)
stop_interval_df = df_to_stop_gdf(stop_interval_df)
stop_interval_df = stop_interval_df.sort_values(stop_id).reset_index(drop=True)
stop_interval_df = stop_interval_df[stop_interval_df.geometry.is_valid]
stop_interval_df

In [ ]:
# Save the interval_df GeoDataFrame to open in QGis 
stop_interval_df.to_file(city_filename+"/stop_interval.gpkg")

In [ ]:
m = stop_interval_df[
    [
        stop_id,
        "mean_interval",
        "stop_name",
        "route_names",
        "route_type_texts",
        "geometry",
    ]
].explore(
    column=f"mean_interval",
    cmap="RdYlGn_r",
    vmin=5,
    vmax=30,
    style_kwds={
        "color": "black",  # Border color
        "weight": 1,  # Border thickness
        "opacity": 1.0,  # Border opacity
        "fillOpacity": 1,
        "radius": 6,
    },
)
m

If you want the interval per **route** and **stop**

In [ ]:
stop_route_interval_df = gtfs.get_mean_interval_at_stops(
    date=selected_day,
    start_time=start_time,
    end_time=end_time,
    route_types=route_types, 
    by = "route_id", # Interval is computed for all 'trip_id' grouped by this column and sorted by 'departure_time'
    at = stop_id, # Where to compute the interval 'stop_id' 'parent_station'
    how = "best", 
    # 'best' pick the route with best interval, 
    # 'mean' Combine all intervals of all routes, 
    # 'all' return results per stop and route
    n_divisions=1, # Number of divisions for by = 'shape_direction'
    # mix_directions=False # For how = 'mean' do you want to consider both directions as different routes?
).to_pandas()
stop_route_interval_df = gtfs.add_stop_coords(stop_route_interval_df)
stop_route_interval_df = gtfs.add_route_names(stop_route_interval_df)
stop_route_interval_df = df_to_stop_gdf(stop_route_interval_df)
stop_route_interval_df = stop_route_interval_df.sort_values(stop_id).reset_index(drop=True)
stop_route_interval_df = stop_route_interval_df[stop_route_interval_df.geometry.is_valid]
stop_route_interval_df

In [ ]:
# Save the interval_df GeoDataFrame to open in QGis 
stop_route_interval_df.to_file(city_filename+"/stop_route_interval.gpkg")

In [ ]:
# Mean interval by route 
if "route_id" not in stop_route_interval_df.columns:
    raise Exception("You should compute get_mean_interval_at_stops with by = 'route_id' to compute route means")

route_interval = (
    stop_route_interval_df
    .groupby('route_id')
    .agg(mean_interval=('mean_interval', 'mean'))
    .reset_index()
)
route_interval = gtfs.add_route_names(route_interval)
route_interval

In [ ]:
route_interval.to_csv(city_filename+"/route_intervals.csv")

In [ ]:
# Mean interval in whole system 
system_interval = float(
    np.nanmean(stop_interval_df['mean_interval'])
)

print(f"The average waiting time (interval) in the system is {round(system_interval,ndigits=2)} minutes")

## Edge data

### Speed

At every stop and by every route 

Speeds are in *km/h*

In [ ]:
edge_speed_df = gtfs.get_mean_speed_at_edges(
    date=selected_day,
    start_time=start_time,
    end_time=end_time,
    route_types = route_types,
    by = "edge_id", # Speed is computed for every 'trip_id' and grouped by this column with the how method
    at = stop_id, # Compute speed for every 'parent_station' 'stop_id' or 'route_id'
    how="mean", # How to group individual trip speeds 'mean' 'max' or 'min'
).to_pandas()
edge_speed_df = gtfs.add_stop_coords(edge_speed_df)
edge_speed_df = gtfs.add_route_names(edge_speed_df)
edge_speed_df = gpd.GeoDataFrame(edge_speed_df,geometry=edge_speed_df['edge_linestring'].apply(wkt.loads),crs=4326)
edge_speed_df = edge_speed_df.sort_values("edge_id").reset_index(drop=True)
edge_speed_df = edge_speed_df[edge_speed_df.geometry.is_valid]
edge_speed_df 

In [ ]:
edge_speed_df.to_file(city_filename+"/edge_speed.gpkg")

In [ ]:
m = edge_speed_df[
    [
        stop_id+"_A",
        stop_id+"_B",
        "speed",
        "stop_name_A",
        "stop_name_B",
        "route_names",
        "route_type_texts",
        "n_trips",
        "geometry",
    ]
].explore(
    column=f"speed",
    cmap="RdYlGn",
    vmin=10,
    vmax=30,
)
m = best_stop_speed_df[
    [
        stop_id,
        "speed",
        "stop_name",
        "route_name",
        "route_type_text",
        "geometry",
    ]
].explore(
    m=m,
    column=f"speed",
    cmap="RdYlGn",
    vmin=10,
    vmax=30,
    style_kwds={
        "color": "black",  # Border color
        "weight": 1,  # Border thickness
        "opacity": 1.0,  # Border opacity
        "fillOpacity": 1,
        "radius": 6,
    },
)
m

In [ ]:
edge_interval_df = gtfs.get_mean_interval_at_edges(
    date=selected_day,
    start_time=start_time,
    end_time=end_time,
    route_types=route_types, 
    by = "edge_id", # Interval is computed for all 'trip_id' grouped by this column and sorted by 'departure_time'
    at = stop_id, # Where to compute the interval 'stop_id' 'parent_station'
    how = "add", 
    # 'best' pick the route with best interval, 
    # 'add' Combine all intervals of all routes, 
    # 'all' return results per stop and route
).to_pandas()
edge_interval_df = gtfs.add_stop_coords(edge_interval_df)
edge_interval_df = gtfs.add_route_names(edge_interval_df)
edge_interval_df = gpd.GeoDataFrame(edge_interval_df,geometry=edge_interval_df['edge_linestring'].apply(wkt.loads),crs=4326)
edge_interval_df = edge_interval_df.sort_values("edge_id").reset_index(drop=True)
edge_interval_df = edge_interval_df[edge_interval_df.geometry.is_valid]
edge_interval_df

In [ ]:
edge_interval_df.to_file(city_filename+"/edge_interval.gpkg")

In [ ]:
m = edge_interval_df[
    [
        stop_id+"_A",
        stop_id+"_B",
        "mean_interval",
        "stop_name_A",
        "stop_name_B",
        "route_names",
        "route_type_texts",
        "n_trips",
        "geometry",
    ]
].explore(
    column=f"mean_interval",
    cmap="RdYlGn_r",
    vmin=5,
    vmax=30,
)
m = stop_interval_df[
    [
        stop_id,
        "mean_interval",
        "stop_name",
        "route_names",
        "route_type_texts",
        "geometry",
    ]
].explore(
    m=m,
    column=f"mean_interval",
    cmap="RdYlGn_r",
    vmin=5,
    vmax=30,
    style_kwds={
        "color": "black",  # Border color
        "weight": 1,  # Border thickness
        "opacity": 1.0,  # Border opacity
        "fillOpacity": 1,
        "radius": 6,
    },
)
m

### Get a dataframe with all the info for your own processing code

In [ ]:
gtfs_df = gtfs.filter(
    date=date,
    start_time=start_time,
    end_time=end_time,
    route_types = route_types,
    frequencies=False,
    in_aoi=True
)
gtfs_df = gtfs.add_route_names(gtfs_df)
gtfs_df = gtfs.add_stop_coords(gtfs_df)
gtfs_df = df_to_stop_gdf(gtfs_df)
gtfs_df = gtfs_df.sort_values(['trip_id','stop_sequence']).reset_index(drop=True)
gtfs_df = gtfs_df[gtfs_df.geometry.is_valid]
gtfs_df

In [ ]:
# Save the complete gtfs_df GeoDataFrame for your own processing
gtfs_df.to_file(city_filename+"/complete_gtfs.gpkg")

# GTFS DataFrame Column Reference

The fields are grouped by theme for clarity.

---

## Core GTFS Identifiers

| Column             | Description                                                                                                                 |
| ------------------ | --------------------------------------------------------------------------------------------------------------------------- |
| **service_id**     | Unique identifier for a service pattern or calendar day. For example, all weekday services may share the same `service_id`. |
| **route_id**       | Unique identifier for each transit line (e.g., Red Line).                                                                   |
| **trip_id**        | Unique identifier for each individual trip (one vehicle going from start to end at specific times).                         |
| **shape_id**       | Unique identifier for a line geometry. Trips sharing the same physical path but different timing use the same `shape_id`.   |
| **direction_id**        | Unique identifier for each route direction [0 or 1]. 
| **stop_id**        | Unique identifier for each stop.                                                                                            |
| **parent_station** | Identifier grouping related stops (e.g., opposite-direction platforms at one station).                                      |

---

## Stop Times

| Column             | Description                                              |
| ------------------ | -------------------------------------------------------- |
| **departure_time** | Departure time from the stop, in seconds after midnight. |
| **arrival_time**   | Arrival time at the stop, in seconds after midnight.     |
| **stop_sequence**  | Ordering of stops within a trip (1 = first stop).        |

---

## Shape / Geometry Details

| Column                        | Description                                                                                                         |
| ----------------------------- | ------------------------------------------------------------------------------------------------------------------- |
| **shape_time_traveled**       | Time elapsed since the first stop, computed from geometry.                                                          |
| **shape_total_travel_time**   | Total computed travel time of the trip.                                                                             |
| **shape_dist_traveled**       | Distance from the starting point of the shape, in meters.                                                           |
| **shape_total_distance**      | Total distance of the trip, in meters.                                                                              |
| **shape_direction**           | Average forward direction of travel at this stop (angle based on this stop and the average of all upcoming stops).  |
| **shape_direction_backwards** | Average backward direction of travel at this stop (angle based on this stop and the average of all previous stops). |

---

## Frequency-Based Scheduling (GTFS-Frequencies)

These columns appear when the GTFS uses frequency-based definitions instead of explicit scheduled trips.

| Column           | Description                                                                                             |
| ---------------- | ------------------------------------------------------------------------------------------------------- |
| **start_time**   | Start of the frequency window, in days since 01-01-1970.                                                |
| **end_time**     | End of the frequency window, same unit as above.                                                        |
| **headway_secs** | Frequency of a repeated trip, in seconds. The trip repeats every `headway_secs` from start to end time. |
| **n_trips**      | Number of trips represented by the frequency specification.                                             |

---

## GTFS Source Metadata

| Column        | Description                                             |
| ------------- | ------------------------------------------------------- |
| **gtfs_name** | Name of the GTFS source file that contributed this row. |
| **file_id**   | Index of the GTFS file in the list of `file_paths`.     |

---

## Route Type

| Column         | Description                                                                                                                                                                                                                                                                                                                |
| -------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **route_type** | GTFS route type, either numeric or string-normalized using your provided function. Valid mapped values: <br><br> **0** – Tram / Streetcar <br> **1** – Subway / Metro <br> **2** – Rail <br> **3** – Bus <br> **4** – Ferry <br> **5** – Cable Car <br> **6** – Gondola / Suspended cable transport <br> **7** – Funicular |

String inputs such as `"tram"`, `"subway"`, `"bus"`, `"cable car"` etc. are automatically normalized.

---

## Spatial Attributes

| Column       | Description                                                                 |
| ------------ | --------------------------------------------------------------------------- |
| **isin_aoi** | Boolean: whether the stop lies inside the defined Area of Interest polygon. |

---